In [1]:
from collections import deque
from datetime import datetime as dt
import heapq
from itertools import combinations
from math import atan2, cos, cosh, radians, sin, sqrt
import os
from pathlib import Path

import dotenv
import googlemaps
import pandas as pd
from tqdm import tqdm

ROOT = Path('.').resolve().parents[0]


In [3]:
df = pd.read_csv(ROOT / 'input/distance_matrix2.csv').drop('Unnamed: 0', axis=1)
df


,0,1,2,3,4,5,6,7,8,9,...,641,642,643,644,645,646,647,648,649,650
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7788
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5479
2,0.0,0.0,0.0,490.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7447
3,0.0,0.0,490.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7923
4,0.0,0.0,0.0,0.0,0.0,0.0,474.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8426.0,0.0,0.0,0.0,0.0,0
647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7799.0,0.0,0.0,0.0,0.0,0
649,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
from collections import defaultdict


class UnionFind():
  """
  Union Find木クラス
  0-index

  Attributes
  --------------------
  n : int
    要素数
  root : list
    木の要素数
    0未満であればそのノードが根であり、添字の値が要素数
  rank : list
    木の深さ
  """

  def __init__(self, n: int) -> None:
    """
    Parameters
    ---------------------
    n : int
      要素数
    """
    self.n = n
    self.root = [-1]*n
    self.rank = [0]*n
    self._size = n

  def find(self, x: int) -> int:
    """
    ノードxの根を見つける

    Parameters
    ---------------------
    x : int
      見つけるノード

    Returns
    ---------------------
    root : int
      根のノード
    """
    if(self.root[x] < 0):
      return x
    else:
      self.root[x] = self.find(self.root[x])
      return self.root[x]

  def unite(self, x: int, y: int) -> None:
    """
    木の併合

    Parameters
    ---------------------
    x : int
      併合したノード
    y : int
      併合したノード
    """
    x = self.find(x)
    y = self.find(y)

    if(x == y):
      return
    elif(self.rank[x] > self.rank[y]):
      self.root[x] += self.root[y]
      self.root[y] = x
    else:
      self.root[y] += self.root[x]
      self.root[x] = y
      if(self.rank[x] == self.rank[y]):
        self.rank[y] += 1

    self._size -= 1

  def same(self, x: int, y: int) -> bool:
    """
    同じグループに属するか判定

    Parameters
    ---------------------
    x : int
      判定したノード
    y : int
      判定したノード

    Returns
    ---------------------
    ans : bool
      同じグループに属しているか
    """
    return self.find(x) == self.find(y)

  def size(self, x: int) -> int:
    """
    木のサイズを計算

    Parameters
    ---------------------
    x : int
      計算したい木のノード

    Returns
    ---------------------
    size : int
      木のサイズ
    """
    return -self.root[self.find(x)]

  def roots(self) -> list[int]:
    """
    根のノードを取得

    Returns
    ---------------------
    roots : list
      根のノード
    """
    return [i for i, x in enumerate(self.root) if x < 0]

  def group_size(self) -> int:
    """
    グループ数を取得

    Returns
    ---------------------
    size : int
      グループ数
    """
    return self._size

  def group_members(self) -> defaultdict[int, list[int]]:
    """
    全てのグループごとのノードを取得

    Returns
    ---------------------
    group_members : defaultdict
      根をキーとしたノードのリスト
    """
    group_members: defaultdict[int, list[int]] = defaultdict(list)
    for member in range(self.n):
      group_members[self.find(member)].append(member)
    return group_members


mat = df.values.tolist()
n = len(mat)
uf = UnionFind(n)
G: list[list[int]] = [[] for _ in range(n)]
for i in range(n):
  for j in range(i+1, n):
    if mat[i][j] != 0 and mat[i][j] != 10**16:
      uf.unite(i, j)
      G[i].append(j)
      G[j].append(i)

deg: set[int] = set()
for i in range(n):
  deg.add(len(G[i]))
print(deg)
print(uf.group_size())


{3, 4, 5, 6, 7, 8}
1
